<a href="https://colab.research.google.com/github/YopaNelly/AI-ML-Basics/blob/main/Translation%20from%20french%20to%20Wolof.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.4 MB/s eta 0:00:00


In [ ]:
!unzip /content/ml-olympiad-machine-translation-french-wolof.zip -d /content/


Archive:  /content/ml-olympiad-machine-translation-french-wolof.zip
  inflating: /content/test.csv       
  inflating: /content/train.csv      


In [ ]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.2 MB/s eta 0:00:00


In [ ]:
from datasets import Dataset


In [ ]:
import evaluate

# Load BLEU metric
bleu_metric = evaluate.load("bleu")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer, Trainer, TrainingArguments
import evaluate

# Load your train and test datasets
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

# Initialize the tokenizer and model
model_name = "Helsinki-NLP/opus-mt-fr-en"  # You might need a custom or retrained French-Wolof model
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
train_data.head()

,ID,FRENCH,WOLOF
0,0,Bonne continuation.,Yal na la jig te yeneeni ndam topp ci.
1,1,Vérifiez que votre main soit détendue le plus ...,Wóo ni sa loxo bi téguna bu baax booy bindee l...
2,2,"Ce renouveau politique, véritable alternative,...","Beesal gi am ci pólótig bi dana tax, ci lu wér..."
3,3,"Mais je ne doute point, vu la mobilisation exc...","Waaye wéddiwuma ko, soo gisee ndaje mu am dool..."
4,4,"Il faut que tu tiennes ta promesse, me dit dou...","Ndoomu buur si toogaat ci sama wet, wax ma ndà..."


In [ ]:
# Preprocess the data
def preprocess_data(examples):
    # Tokenize French input
    inputs = tokenizer(examples["FRENCH"], max_length=128, truncation=True, padding="max_length")
    # Tokenize Wolof output (label)
    targets = tokenizer(examples["Wolof"], max_length=128, truncation=True, padding="max_length")

    inputs["labels"] = targets["input_ids"]
    return inputs

# Convert the train and test data into Hugging Face datasets
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)

# Apply the preprocessing function
train_dataset = train_dataset.map(preprocess_data, batched=True)
test_dataset = test_dataset.map(preprocess_data, batched=True)


Map:   0%|          | 0/7263 [00:00<?, ? examples/s]

KeyError: 'Wolof'

In [ ]:
def preprocess_data(examples):
    # Adjust these keys based on your actual column names
    french_texts = examples["FRENCH"]
    wolof_texts = examples["WOLOF"]

    model_inputs = tokenizer(french_texts, max_length=128, truncation=True, padding="max_length")

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(wolof_texts, max_length=128, truncation=True, padding="max_length")
        model_inputs["labels"] = labels["input_ids"]

    return model_inputs


In [ ]:
# Apply the preprocessing function
train_dataset = train_dataset.map(preprocess_data, batched=True)
test_dataset = test_dataset.map(preprocess_data, batched=True)


Map:   0%|          | 0/7263 [00:00<?, ? examples/s]

Map:   0%|          | 0/2270 [00:00<?, ? examples/s]

KeyError: 'WOLOF'

In [ ]:
# Import required libraries
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer, Trainer, TrainingArguments
import evaluate

# Load and inspect the training and test datasets
train_data = pd.read_csv("train.csv")  # Adjust with your train CSV path
test_data = pd.read_csv("test.csv")    # Adjust with your test CSV path

# Ensure the train dataset has FRENCH and WOLOF columns, while the test dataset only has FRENCH
print("Training data columns:", train_data.columns)
print("Testing data columns:", test_data.columns)

# Convert pandas dataframes to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)

# Load the MarianMT tokenizer and model for French to Wolof translation
model_name = "Helsinki-NLP/opus-mt-fr-en"  # Starting with French-to-English, to be fine-tuned
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Preprocessing function for the train dataset, which includes target labels (WOLOF)
def preprocess_train_data(examples):
    # Tokenize the French input
    model_inputs = tokenizer(examples["FRENCH"], max_length=128, truncation=True, padding="max_length")
    # Tokenize the Wolof target labels
    model_inputs["labels"] = tokenizer(text_target=examples["WOLOF"], max_length=128, truncation=True, padding="max_length")["input_ids"]
    return model_inputs

# Preprocessing function for the test dataset, which only includes source text (FRENCH)
def preprocess_test_data(examples):
    # Tokenize only the French input for testing
    return tokenizer(examples["FRENCH"], max_length=128, truncation=True, padding="max_length")

# Apply preprocessing to train and test datasets
train_dataset = train_dataset.map(preprocess_train_data, batched=True)
test_dataset = test_dataset.map(preprocess_test_data, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Start training
trainer.train()

# Evaluate the model
trainer.evaluate()

# Define a function to translate new French sentences into Wolof
def translate_text(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    translated_tokens = model.generate(**inputs)
    return tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

# Example: Test translation with a sample French sentence
french_text = "Bonne continuation."
wolof_translation = translate_text(french_text)
print(f"French: {french_text}")
print(f"Wolof: {wolof_translation}")


Training data columns: Index(['ID', 'FRENCH', 'WOLOF'], dtype='object')
Testing data columns: Index(['ID', 'FRENCH'], dtype='object')


/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/7263 [00:00<?, ? examples/s]

Map:   0%|          | 0/2270 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
